<a href="https://colab.research.google.com/github/NatalieGo/MFTI_hackathon_1/blob/main/%D0%A0%D0%B0%D0%B7%D0%B1%D0%BE%D1%80_json_isp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from IPython.display import display, HTML
data = pd.read_json('data/RU_dataset.json')
display(data)
display(data.info())

,0,1
0,Бывшая жена угрожает самоубийством Недавно я н...,suicide
1,"Я странный? На меня не влияют комплименты, есл...",non-suicide
2,Наконец-то 2020 год почти закончился... Так чт...,non-suicide
3,"мне нужна помощь, просто помоги мне, я так плачу",suicide
4,"Я так растеряна. Здравствуйте, меня зовут Адам...",suicide
...,...,...
47992,"Я только что увидел забавный мем, прежде чем п...",suicide
47993,"Я боюсь, мне некому рассказать. У меня инфекци...",non-suicide
47994,"Педофилия, членовредительство и мысли о самоуб...",suicide
47995,Я больше не могу справляться. Почти 2 месяца н...,suicide


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47997 entries, 0 to 47996
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       47997 non-null  object
 1   1       47997 non-null  object
dtypes: object(2)
memory usage: 750.1+ KB


None

В данном датасете информация о 47997 сообщениях с Raddit, которые размечены по 2м категориям: «non-suicide» и «suicide»
Содержит 47997 строк, 2 столбца («0» и «1» ), оба столбца имеют тип object. для того, что бы оценить наличие лишних символов и корректность разметки, преобразуем файл в CSV формат. Так же переименуем столбцы.

In [ ]:
data = data.rename(columns={0: 'Сообщение', 1: 'Самоубийство'})
display(data)


,Сообщение,Самоубийство
0,Бывшая жена угрожает самоубийством Недавно я н...,suicide
1,"Я странный? На меня не влияют комплименты, есл...",non-suicide
2,Наконец-то 2020 год почти закончился... Так чт...,non-suicide
3,"мне нужна помощь, просто помоги мне, я так плачу",suicide
4,"Я так растеряна. Здравствуйте, меня зовут Адам...",suicide
...,...,...
47992,"Я только что увидел забавный мем, прежде чем п...",suicide
47993,"Я боюсь, мне некому рассказать. У меня инфекци...",non-suicide
47994,"Педофилия, членовредительство и мысли о самоуб...",suicide
47995,Я больше не могу справляться. Почти 2 месяца н...,suicide


In [ ]:
#Идентификация пропущенных значений (NaN, NULL, пустых строк и т. д.) в датафрейме
filtered_colums = data.eq("").sum()
display(filtered_colums)
print(data.info())
data.isnull().sum()

Сообщение       0
Самоубийство    0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47997 entries, 0 to 47996
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Сообщение     47997 non-null  object
 1   Самоубийство  47997 non-null  object
dtypes: object(2)
memory usage: 750.1+ KB
None


Сообщение       0
Самоубийство    0
dtype: int64

In [ ]:
#Посчитаем сколько постов относятся к  «Пост о потенциальном самоубийстве»
data['Самоубийство'].value_counts()

Самоубийство
non-suicide    24081
suicide        23916
Name: count, dtype: int64

Посчитаем сколько постов сигнализируют о риске суицида, получаем что : «non-suicide» - 24081  и «suicide» - 23916. Видим равномерное распределение. Это плюс для нашей модели. Удаляем лишние символы. Удаляем неинформативные строки, состоящие только из цифр и длинной менее 5 символов. На этом этапе выводим 50 первых строк для оценки корректности разметки.

In [ ]:
# используем IPython.display для красивого вывода всего текста в таблице
from IPython.display import display, HTML
data['Сообщение'] = data['Сообщение'].replace(r'http[s]?://\S+|@\w+|/|&\w+|ð\w+|¶+|RT|aâ¦|@|w\w+|:|#\w+|_|-|ð|ó|¦|§|®|¥|¢|´|¿|Ã¯ÂÂ½', '', regex=True)
data['Сообщение'] = data['Сообщение'].str.replace(r'^\,+', '', regex=True)
data['Сообщение'] = data['Сообщение'].str.replace(r'\n', '', regex=True)
data['Сообщение'] = data['Сообщение'].str.replace(r'XD', '', regex=True)
data['Сообщение'] = data['Сообщение'].str.replace(r'<3', '', regex=True)
data['Сообщение'] = data['Сообщение'].str.lower()
data['Сообщение'] = data['Сообщение'].str.lstrip()
data = data[data['Сообщение'].apply(lambda x: len(x) > 5)] #удаляем неинформативные строки длиной мененее пяти символов
data = data[data['Сообщение'].apply(lambda x: x.isdigit() == False)] #удаляем строки состоящие только из цифр
display(HTML(data.tail(10).to_html()))

,Сообщение,Самоубийство
47987,"хорошо, я пойду спать. надеюсь, сегодня вечером мне не приснятся сны, невесело просыпаться от своего",non-suicide
47988,"я много думал о том, чтобы покончить с этим. в последнее время я действительно думал о том, как было бы здорово умереть и уйти от жизни. последние несколько дней было очень плохо, но сегодня вечером я поставил перед собой крошечную цель — сделать определенное количество отжиманий. это действительно помогло мне, даже если это всего лишь минутная отсрочка. я просто записал то, что чувствовал, и не позволил себе уйти, пока не закончил. я уверен, что это сработает не для всех, но надеюсь, что ктоnenнибудь увидит это, сможет найти некоторый контроль и чтоnenто почувствовать. я надеюсь, что любой, кто борется, получит некоторое облегчение. у вас есть ценность, и вы имеете значение.",suicide
47989,"это отстой, что есть пандемия... но я не буду лгать и говорить, что не хочу, чтобы это дало мне выход.мол, это отстой и больно, но было бы так удобно. не моя вина. это не вина моей семьи. просто невинная жертва. одна из тысяч невинных жертв. просто еще один номер. исчезните в новостях как еще одна безликая безымянная отметка в блокноте. и я обрету покой. ушло, ни завтра, ни послезавтра. мои родители не будут винить себя, они будут оплакивать беспомощность ситуации и, вероятно, просто посвятят себя помощи другим жертвам.на мои похороны никто не придет.мои родители знают, что у меня ноль друзей. они сделают приглашение в facebook, и никто не придет. даже мои сестры будут счастливее, они на самом деле меня ненавидят. когда моя сестра вышла замуж, у нее было 8 подружек невесты. если бы я когдаnenнибудь женился... мне бы некого было пригласить, потому что у меня нет друзей. пару лет назад я устроил себе вечеринку по случаю дня рождения, и никто не пришел.это моя вина. меня зовут карен, и я девушка, которую ты ненавидел. я раздражающая, красивая, но недостаточно красивая, неуклюжая, неуверенная в себе. я сдался. я сдаюсь.просто покончим с этим. я закончил попытки. я покончил с сбоями, перезапусками и ни за что не цепляясь. дайте мне чтоnenнибудь, на что можно улыбнуться, указать и сказатьnen «смотри! я трахаюсь! оно выбрало правильно! до свидания!""",suicide
47990,"помоги пожалуйста. мне только прислали одну из этих хакшин, помогите.",non-suicide
47991,"кто, черт возьми, не хочет покончить с собой? все, кого я знаю, испытывают некоторую степень депрессии или беспокойства, даже если они не признаются в этом открыто; это почти стало скорее правилом, чем исключением. я больше не испытываю никакой жалости к тем, кто заявляет о симптомах депрессии, потому что на самом деле это означает следующееnen «я думающий человек, достаточно самосознательный, чтобы понимать, что быть живым — это чертовски отстойно».я думал о смерти большую часть двух лет и не могу себе представить, чтобы некоторые из этих людей держались так чертовски долго. в чем смысл? зачем оставаться в живых? только потому, что ты причинишь вред всем остальным? это чертовски глупо. с меня хватит этого дерьма, да и с тебя, наверное, тоже, даже если ты пытаешься поручиться на всю жизнь.",suicide
47992,"я только что увидел забавный мем, прежде чем покончить с собой. вероятно, это был мем высшего уровня, вероятно, потому, что он заставил меня смеяться сквозь слезы. теперь я мог бы с таким же успехом сказать, что умер счастливым. какой эпический способ покончить с жизнью. естественный отбор делает свое дело. в любом случае, спасибо reddit за то, что это отличное место, где я иногда могу высказаться, и до свидания!",suicide
47993,"я боюсь, мне некому рассказать. у меня инфекция ногтей, и это нормально, но все прошло совсем плохо. дальше кончика оно не вышло, но сильно опухло, болит и немного желтеет под ногтем. я даже ручку держать не могу, а через неделю с небольшим мне предстоит сдать целый месяц экзаменов, я не знаю, как я буду чтоnenто делать, я даже проснулась до 6 утра, потому что больно. прошла неделя. извините, если это было наглядно, мне н

In [ ]:
#Снова посчитаем данные о количестве постов
data['Самоубийство'].value_counts()

Самоубийство
non-suicide    24080
suicide        23915
Name: count, dtype: int64

Осталось 24077 сообщений не сигнализирующих о самоубийстве и 23914 сообщений сигнализирующих о самоубийстве. Сохраним новый вариант дата сета.

In [ ]:
data.to_csv('data/ru_Sui_new_isp_5.csv', index=False, sep='\t')